第二步：准备你的数据集
确保你的图像数据被组织成适当的文件夹结构。通常，我们会有一个根文件夹，里面包含两个子文件夹：train 和 val（分别用于训练和验证）。每个子文件夹下应该包含代表类别的进一步子文件夹，每个类别文件夹里包含相应的图像文件。

data/
    train/
        class1/
            img1.jpg
            img2.jpg
            ...
        class2/
            img1.jpg
            img2.jpg
            ...
    val/
        class1/
            img1.jpg
            img2.jpg
            ...
        class2/
            img1.jpg
            img2.jpg
            ...


第三步：加载数据
使用 PyTorch 的数据加载器和数据变换工具来准备你的数据。这包括规范化图像尺寸和颜色值，使其适合 ResNet50 模型。

In [1]:
from torchvision import transforms, datasets
import torch
import os
# 设置数据转换
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 数据目录
data_dir = r'F:\AI\resnet50\data'

# 加载数据集
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}


第四步：修改并设置 ResNet50 模型
加载预训练的 ResNet50 模型，并修改它的最后一层，以适应你的类别数量。

In [3]:
from torchvision import models

model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(image_datasets['train'].classes))


第五步：配置训练过程
设置优化器和损失函数，准备模型训练。

In [4]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


第六步：训练模型
编写代码来训练模型，记录训练和验证过程中的损失和准确率。

In [6]:
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

# 调用训练函数
model = train_model(model, criterion, optimizer, num_epochs=25)


Epoch 0/24
----------
train Loss: 0.1861 Acc: 0.9258
val Loss: 0.0031 Acc: 1.0000
Epoch 1/24
----------
train Loss: 0.4201 Acc: 0.8477
val Loss: 0.0000 Acc: 1.0000
Epoch 2/24
----------
train Loss: 0.2063 Acc: 0.9297
val Loss: 0.0000 Acc: 1.0000
Epoch 3/24
----------
train Loss: 0.2345 Acc: 0.9297
val Loss: 0.0001 Acc: 1.0000
Epoch 4/24
----------
train Loss: 0.2841 Acc: 0.9297
val Loss: 0.0000 Acc: 1.0000
Epoch 5/24
----------
train Loss: 0.3153 Acc: 0.8945
val Loss: 0.0000 Acc: 1.0000
Epoch 6/24
----------
train Loss: 0.1638 Acc: 0.9297
val Loss: 0.0003 Acc: 1.0000
Epoch 7/24
----------
train Loss: 0.1462 Acc: 0.9492
val Loss: 0.0001 Acc: 1.0000
Epoch 8/24
----------
train Loss: 0.0133 Acc: 0.9961
val Loss: 0.0010 Acc: 1.0000
Epoch 9/24
----------
train Loss: 0.0586 Acc: 0.9844
val Loss: 0.0000 Acc: 1.0000
Epoch 10/24
----------
train Loss: 0.0596 Acc: 0.9805
val Loss: 0.0001 Acc: 1.0000
Epoch 11/24
----------
train Loss: 0.1896 Acc: 0.9453
val Loss: 0.0000 Acc: 1.0000
Epoch 12/24
--

第七步：保存和评估模型
在训练结束后，保存你的模型，并在新的数据上进行测试以评估其性能。

In [7]:
torch.save(model.state_dict(), 'model_resnet50.pth')
